## Fake NEws Classificer USsing Bidirectional LSTM

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
## Dataset: https://www.kaggle.com/competitions/fake-news/data?select=train.csv
import pandas as pd

In [31]:
df=pd.read_csv('/content/drive/MyDrive/Data Science Files/fake news data/fake-news/train.csv')#,engine='python',error_bad_lines=False)

In [32]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [33]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [34]:
df.shape

(20800, 5)

In [35]:
df=df.dropna()

In [36]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [37]:
## get the independent features
X=df.drop('label',axis=1)

In [38]:
y=df['label']

In [39]:
## Check whether dataset is balanced or not
y.value_counts()

0    10361
1     7924
Name: label, dtype: int64

In [40]:
import tensorflow as tf

In [41]:
tf.__version__

'2.9.2'

In [42]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional


In [43]:
## vocabulary size
voc_size=5000

In [44]:
messages=X.copy()

In [45]:
messages.head(10)

,id,title,author,text
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...
5,5,Jackie Mason: Hollywood Would Love Trump if He...,Daniel Nussbaum,"In these trying times, Jackie Mason is the Voi..."
7,7,Benoît Hamon Wins French Socialist Party’s Pre...,Alissa J. Rubin,"PARIS — France chose an idealistic, traditi..."
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...
10,10,Obama’s Organizing for Action Partners with So...,Aaron Klein,"Organizing for Action, the activist group that..."
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...


In [46]:
import nltk
import re
from nltk.corpus import stopwords

In [47]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
messages.reset_index(inplace=True)

In [49]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    print(i)
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

Streaming output truncated to the last 5000 lines.
13285
13286
13287
13288
13289
13290
13291
13292
13293
13294
13295
13296
13297
13298
13299
13300
13301
13302
13303
13304
13305
13306
13307
13308
13309
13310
13311
13312
13313
13314
13315
13316
13317
13318
13319
13320
13321
13322
13323
13324
13325
13326
13327
13328
13329
13330
13331
13332
13333
13334
13335
13336
13337
13338
13339
13340
13341
13342
13343
13344
13345
13346
13347
13348
13349
13350
13351
13352
13353
13354
13355
13356
13357
13358
13359
13360
13361
13362
13363
13364
13365
13366
13367
13368
13369
13370
13371
13372
13373
13374
13375
13376
13377
13378
13379
13380
13381
13382
13383
13384
13385
13386
13387
13388
13389
13390
13391
13392
13393
13394
13395
13396
13397
13398
13399
13400
13401
13402
13403
13404
13405
13406
13407
13408
13409
13410
13411
13412
13413
13414
13415
13416
13417
13418
13419
13420
13421
13422
13423
13424
13425
13426
13427
13428
13429
13430
13431
13432
13433
13434
13435
13436
13437
13438
13439
13440
13441
13442
1

In [50]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [51]:
onehot_repr=[one_hot(words,voc_size) for words in corpus]
onehot_repr

[[4756, 2175, 4342, 1312, 2474, 462, 4284, 2750, 2402, 2415],
 [223, 4869, 4659, 662, 583, 4836, 3492],
 [636, 561, 3845, 1111],
 [3690, 4018, 283, 4628, 3926, 515],
 [652, 583, 2514, 4665, 2927, 3917, 583, 388, 3860, 896],
 [2760,
  2886,
  4767,
  648,
  1670,
  2875,
  1058,
  2489,
  432,
  4487,
  1139,
  1166,
  1308,
  2772,
  3492],
 [4492, 715, 2861, 2850, 4173, 4012, 2892, 4150, 1270, 2899, 1245],
 [3449, 1334, 1095, 4059, 1347, 4890, 2875, 4960, 1270, 2899, 1245],
 [1197, 301, 4408, 1319, 54, 1523, 4304, 3017, 2875, 1077],
 [3792, 3914, 1163, 4025, 575, 89, 2623, 4404],
 [3768, 4343, 4985, 1088, 3605, 1718, 2063, 2557, 3707, 854, 3403],
 [4628, 3015, 2474, 1523, 2875, 1347],
 [2225, 411, 4980, 1743, 2343, 3052, 523, 615, 3080],
 [282, 359, 2673, 2499, 675, 472, 4779, 1270, 2899, 1245],
 [3209, 3146, 2179, 3009, 528, 1270, 2899, 1245],
 [4364, 1342, 173, 1014, 4072, 2805, 3159, 2974, 2452, 1048],
 [1708, 3499, 4869],
 [4134, 4349, 3124, 440, 2875, 902, 4168, 3492],
 [564, 254

## Embedding Representation

In [52]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

embedded_docs


array([[   0,    0,    0, ..., 2750, 2402, 2415],
       [   0,    0,    0, ...,  583, 4836, 3492],
       [   0,    0,    0, ...,  561, 3845, 1111],
       ...,
       [   0,    0,    0, ..., 1270, 2899, 1245],
       [   0,    0,    0, ...,  889, 2658,  965],
       [   0,    0,    0, ..., 3581, 2425,   78]], dtype=int32)

In [53]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_1 (Dense)             (None, 1)                 201       
                                                                 
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [55]:
## train test split

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [56]:
## Model Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=32)

Epoch 1/10
383/383 [==============================] - 12s 11ms/step - loss: 0.2615 - accuracy: 0.8807 - val_loss: 0.1912 - val_accuracy: 0.9196
Epoch 2/10
383/383 [==============================] - 3s 9ms/step - loss: 0.1260 - accuracy: 0.9490 - val_loss: 0.2015 - val_accuracy: 0.9218
Epoch 3/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0759 - accuracy: 0.9733 - val_loss: 0.2642 - val_accuracy: 0.8996
Epoch 4/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0442 - accuracy: 0.9847 - val_loss: 0.2843 - val_accuracy: 0.9112
Epoch 5/10
383/383 [==============================] - 3s 9ms/step - loss: 0.0256 - accuracy: 0.9922 - val_loss: 0.3555 - val_accuracy: 0.9153
Epoch 6/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0162 - accuracy: 0.9953 - val_loss: 0.4633 - val_accuracy: 0.9095
Epoch 7/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0096 - accuracy: 0.9972 - val_loss: 0.4387 - val_accuracy: 0.9100
Ep

In [57]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 3ms/step


In [58]:
import numpy as np

y_pred=np.where(y_pred>=0.5,1,0)

In [59]:
from sklearn.metrics import confusion_matrix,accuracy_score
confusion_matrix(y_test,y_pred)

array([[3063,  356],
       [ 157, 2459]])

In [60]:
print(accuracy_score(y_test,y_pred))

0.9149958574979288
